# Using a Particle Filter for Feedback Control

In this tutorial, we will give a brief example of how to use a Particle Filter from `ParticleFilters.jl` for feedback control.

## System Description

The sysem is a two-dimensional discrete-time double integrator with gaussian process and observation noise. The state at time $k$, $x_k$, is a 4-element vector consisting of the position in two dimensions and the velocity. The dynamics can be represented with the linear difference equation:

$$x_{k+1} = f(x_k, u_k, w_k) = A x_k + B u_k + w_k \text{,}$$

where $w_k$ are independent but identically-distributed zero-mean Gaussian random variables with covariance matrix $W$. $A$ and $B$ are given in the code below.

The measurements are noisy observations of the position generated according to 

$$y_{k+1} = h(x_k, u_k, x_{k+1}, v_{k+1}) = C x_{k+1} + v_{k+1} \text{,}$$

where $C$ selects only the position of the model, and $v_k$ are independent identically-distributed zero-mean Gaussian random variables with covariance matrix $V$.

## Control Law Description

The control law, will use the mean from the particle filter belief to try to make the state oscillate about the origin, i.e.

$$u = K \hat{x}$$

where $\hat{x}$ is the mean estimate of the state, and

$$K = [-1, -1, 0, 0]$$

It should be noted that, since this system is linear with Gaussian noise, a [Kalman Filter](https://en.wikipedia.org/wiki/Kalman_filter) would be much better-suited for this case, but we use the system here for its simplicity. In general, particle filters can be used with any nonlinear dynamical systems.

In [1]:
using ParticleFilters
using Distributions
using StaticArrays
using LinearAlgebra
using Random

## Dynamics model

We begin by defining the dynamics of the system. For more information about defining a model, see the "Models" section of the docs [TODO: add link].

In [2]:
const dt = 0.1 # time step

const A = [1.0 0.0 dt  0.0;
           0.0 1.0 0.0 dt ;
           0.0 0.0 1.0 0.0;
           0.0 0.0 0.0 1.0]

const B = [0.5*dt^2 0.0     ;
           0.0      0.5*dt^2;
           dt       0.0     ;
           0.0      dt      ]

const W = Matrix(0.01*Diagonal{Float64}(I, 4)) # Process noise covariance
const V = Matrix(Diagonal{Float64}(I, 2)) # Measurement noise covariance

f(x, u, rng) = A*x + B*u + rand(rng, MvNormal(W))

f (generic function with 1 method)

## Observation Model

Next, the observation model is defined. $h$ generates an observation (this is only used in the simulation, not in the particle filter), $g$ returns the relative likelyhood of an observation given the previous state, control, and current state.

In [3]:
h(x, rng) = rand(rng, MvNormal(x[1:2], V))
g(x0, u, x, y) = pdf(MvNormal(x[1:2], V), y)

g (generic function with 1 method)

## Particle Filter

These models are combined to create a model suitable for the filter. Note that the type of the state is designated as a parameter to the constructor. See the "Models" section of the documentation for more info [TODO: add link].

In [4]:
model = ParticleFilterModel{Vector{Float64}}(f, g)

ParticleFilterModel{Array{Float64,1},typeof(f),typeof(g)}(f, g)

and the particle filter itself is created.

In [5]:
N = 1000
filter = SIRParticleFilter(model, N);

## Running a Simulation

To run a simulation, first an initial belief and initial state need to be created. The initial beleif will consist of particles selected uniformly from $[-2, 2]^4$.

The simulation consists of a loop in which the control is calculated from the mean of the particle. Then the state is updated with the dynamics and a new measurement is generated. Finally, the filter is used to update the belief based on this new measurement.

Using the `update` function in this interactive fashion allows the particle  filter to be used as an estimator/observer for feedback control.

In [6]:
b = ParticleCollection([4.0*rand(4).-2.0 for i in 1:N])
x = [0.0, 1.0, 1.0, 0.0];

In [7]:
using Plots
rng = Random.GLOBAL_RNG;

┌ Info: Recompiling stale cache file /home/zach/.julia/compiled/v1.0/Plots/ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1184
  likely near /home/zach/.julia/packages/Plots/Ufx0i/src/Plots.jl:8


In [13]:
plots = []

for i in 1:100
    print(".")
    m = mean(b)
    u = [-m[1], -m[2]] # Control law - try to orbit the origin
    x = f(x, u, rng)
    y = h(x, rng)
    b = update(filter, b, u, y)

    plt = scatter([p[1] for p in particles(b)], [p[2] for p in particles(b)], color=:black, markersize=0.1, label="")
    scatter!(plt, [x[1]], [x[2]], color=:blue, xlim=(-5,5), ylim=(-5,5), label="")
    push!(plots, plt)
end

....................................................................................................

### Visualization

The results of this simulation can be visualized as a gif animation.

In [9]:
using Reel

In [10]:
frames = Frames(MIME("image/png"), fps=10)
for plt in plots
    print(".")
    push!(frames, plt)
end
write("output.gif", frames)

....................................................................................................

"output.gif"

In [11]:
HTML("""<img src="output.gif"/>""")

HTML{String}("<img src=\"output.gif\"/>")